# Solenoid Modeling

In [ ]:
from pmd_beamphysics.fields.solenoid import make_solenoid_fieldmesh

from pmd_beamphysics.fields.analysis import check_static_div_equation
from pmd_beamphysics.units import mu_0

from pmd_beamphysics import FieldMesh

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
FM = make_solenoid_fieldmesh(
    radius=0.05,
    L=0.2,
    rmax=0.1,
    zmin=-0.4,
    zmax=0.4,
    nr=101,
    nz=200,
    nI=1,
)

In [ ]:
FM.plot()

In [ ]:
FM.plot_onaxis()

In [ ]:
check_static_div_equation(FM, rtol=1e-2, plot=True)

## Hard edge

Making the radius very small approximates a hard-edge model.

Here we expext that $B_z = \mu_0 n I$

In [ ]:
FM_hard = make_solenoid_fieldmesh(
    radius=1e-9,
    L=0.2,
    rmax=0.1,
    zmin=-0.4,
    zmax=0.4,
    nr=101,
    nz=200,
    nI=1,
)

In [ ]:
FM_hard.Bz[0, 0, :].max() == mu_0

In [ ]:
FM_hard.plot_onaxis()

## Compare with a real solenoid

In [ ]:
FM2 = FieldMesh("../data/solenoid.h5")
FM2.plot_onaxis()

In [ ]:
FM3 = make_solenoid_fieldmesh(
    radius=0.0191,
    L=0.02936 * 2,
    rmax=0.1,
    zmin=-0.1,
    zmax=0.1,
    nr=100,
    nz=40,
    nI=1,
)

In [ ]:
fig, ax = plt.subplots()
z0, Bz0 = FM2.axis_values("z", "Bz")
Bz0 = np.real(Bz0 / Bz0.max())
ax.plot(z0, Bz0, label="Superfish")

z, Bz = FM3.axis_values("z", "Bz")
Bz = np.real(Bz / Bz.max())
ax.plot(z, Bz, label="Ideal")
plt.legend()
ax.set_ylim(0, None)
ax.set_xlabel(r"$z$ (m)")
ax.set_ylabel(r"B_z (T)")

In [ ]:
FM2.plot()
FM3.plot()

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt


# Define the Bz model from the given formula
def Bz_model(z, a, b):
    """
    Normalized ideal on-axis model
    """
    A = np.hypot(a, b) / (2 * b)
    term1 = (z + b) / np.hypot(z + b, a)
    term2 = (z - b) / np.hypot(z - b, a)
    return A * (term1 - term2)


z_data = z0
Bz_data = Bz0

# Normalize the data
Bz_data = Bz_data / np.max(Bz_data)  # Normalize Bz to a maximum of 1
z_data = z_data - z_data[np.argmax(Bz_data)]  # Shift z so maximum is at z=0

# Fit the data to the model
popt, pcov = curve_fit(Bz_model, z_data, Bz_data, p0=[0.1, 0.1])

# Extract fitted parameters
a_fit, b_fit = popt
a_fit = abs(a_fit)
b_fit = abs(b_fit)
print(f"Fitted parameters: a = {a_fit}, b = {b_fit}")

# Plot the data and the fitted curve
z_fit = np.linspace(np.min(z_data), np.max(z_data), 500)
Bz_fit = Bz_model(z_fit, a_fit, b_fit)

plt.figure(figsize=(8, 6))
plt.scatter(z_data, Bz_data, label="Normalized Data", color="blue", alpha=0.7)
plt.plot(z_fit, Bz_fit, label="Fitted Model", color="red", linewidth=2)
plt.xlabel("z (normalized)")
plt.ylabel("Bz (normalized)")
plt.title("Fitting Bz Model to Data")
plt.legend()
plt.grid()
plt.show()